In [ ]:
%load_ext autoreload
%autoreload 2

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import geopandas as gpd
import pandas as pd
import json
import folium
from folium import Map, Polygon
from shapely import intersection_all
from shapely.geometry import mapping
import shapely

import numpy as np

In [ ]:
# aoi = Vector('/data/aoi_bretagne.geojson')
# infiles = [
#     "/data/S1/S1A_IW_SLC__1SDV_20230826T180527_20230826T180554_050050_0605B2_E768.zip",
#     "/data/S1/S1A_IW_SLC__1SDV_20230814T180526_20230814T180553_049875_05FFB4_7D92.zip",
# ]
file_aoi = "/data/aoi_bretagne.geojson"
aoi = geojson_to_wkt(read_geojson(file_aoi)["features"][0])
with open("/data/creds.json") as f:
    creds = json.load(f)
api = SentinelAPI(creds["username"], creds["password"])
products = api.query(
    aoi,
    producttype="SLC",
    date=("20220110", "20220430"),
    #  orbitdirection='ASCENDING',
    limit=50,
)
gdf = api.to_geodataframe(products)
gdf_ = gdf.loc[:, ["title", "geometry", "summary", "relativeorbitnumber", "slicenumber"]].reset_index()

In [ ]:
map_ = Map()
grouped = gdf.groupby(["relativeorbitnumber", "slicenumber"])
groups = list(grouped.groups.keys())
for g in groups:
    # convert Multipolygons to Polygons
    dfg = grouped.get_group(g).explode(index_parts=False).to_crs("EPSG:4236")
    # find the common part of all geometries
    inter = shapely.ops.transform(lambda x, y: (y, x), intersection_all(dfg.geometry))
    folium_geo = Polygon(mapping(inter)["coordinates"], style_function=lambda x: {"fillColor": "orange"})
    folium.Tooltip(dfg['summary']).add_to(folium_geo)
    folium_geo.add_to(map_)
    # print(inter)
map_